In [ ]:
import pycuda.autoinit
from pycuda.compiler import SourceModule
import pycuda.driver as drv
import numpy as np
import matplotlib.pyplot as plt

# Leer Kernel

In [ ]:
# Lee el archivo kernel.cu
with open("../src/burger.cu", "r") as f:
    kernel_code = f.read()
mod = SourceModule(kernel_code)    
# Accede a las funciones del kernel
Lax_Friedrichs = mod.get_function("Lax_Friedrichs")
maxIni = mod.get_function("maxReductionIniKernel")
max_fin = mod.get_function("maxReductionKernelEnd")
maximo = mod.get_function("maxReductionKernel")
get_max = mod.get_function("getMaxKernel")

# Configurar las condiciones iniciales del PVI y parámetros de la EDP

In [ ]:
T = np.float64(0.5)
L = np.float64(2)  

# Configurar parámetros de método numérico

In [ ]:
n = np.int32((1<<17))
cfl = np.float64(0.8)
# Creando Condición Inicial
data = np.zeros(2*n,dtype=np.float64)
data[0:int(n/4)] = [np.float64(1.0+i/n) for i in range(0, int(n/4))]
data[0:int(n/4)] = np.float64(1.0) 
cond_ini = data[:n] # para plotear 

dx = np.float64(L / (n))            # Paso espacial (dominio de x de 0 a L)
dt =  np.float64(cfl*dx/np.max(data))  # Paso temporal inicial
dt_dx = np.float64(dt/dx )

# Configurar parámetros GPU

In [ ]:
block_size = (1024, 1, 1)
grid_size = (int(np.ceil(2*n / block_size[0])), 1, 1)

# Inicialización vectories máximo
max_arr = np.zeros(int(np.ceil(2*n/block_size[0])),dtype=np.float64)
max_end = np.zeros(1,dtype=np.float64)

#Asignar memoria en el device
data_gpu = drv.mem_alloc(data.nbytes)
max_arr_gpu = drv.mem_alloc(max_arr.nbytes)
max_end_gpu = drv.mem_alloc(max_end.nbytes)


#Copiar los datos del host al device
drv.memcpy_htod(data_gpu, data)
drv.memcpy_htod(max_arr_gpu, max_arr)
drv.memcpy_htod(max_end_gpu, max_end)

In [ ]:
print("Parámetros EDP:")
print(f"  - Tiempo de simulación: {T} s")
print(f"  - Largo del dominio: {L} s")
print("\nParámetros Método Numérico:")
print(f"  - CFL: {cfl}")
print(f"  - Mallado: {n} celdas")
print(f"  - dx: {dx} [s]")
print(f"  - dt: {dt} [m]")
print("\nParámetros GPU:")
print(f"  - Memoria: {data.nbytes // (1024.0 ** 2)} MB")
print(f"  - Tamaño bloque: {block_size[0]} hilos")
print(f"  - Tamaño grilla: {grid_size[0]} bloques")

# Cálculo de la solución usando el Kernel de CUDA 

In [ ]:
try:
    t=np.int32(0)
    T_actual = 0
    while T_actual <T:
        #----------------- Actualización de data -----------------#
        Lax_Friedrichs(data_gpu, dt_dx, n, t, block=block_size, grid=grid_size)
        pycuda.autoinit.context.synchronize()  # Sincronizar después de la primera fase
        #----------------- Cálculo de MAXIMO para CFL-----------------#
        largo = np.int32(2*n)
        grid_max = grid_size[0]
        maxIni(data_gpu,max_arr_gpu,np.int32(largo), block=block_size, grid=(grid_max,1,1),shared = block_size[0]*max_arr[0].nbytes)
        pycuda.autoinit.context.synchronize()  # Sincronizar después de la primera fase
            
        while grid_max > 1:
            largo = np.int32(largo/block_size[0])
            grid_max = int(np.ceil(grid_max / block_size[0]))
            maximo(max_arr_gpu,np.int32(largo), block=block_size, grid=(grid_max,1,1),shared = block_size[0]*max_arr[0].nbytes)
            pycuda.autoinit.context.synchronize()  # Sincronizar después de la primera fase
        get_max(max_arr_gpu,max_end_gpu, largo, block=(int(largo),1,1), grid=(1,1,1),shared = block_size[0]*max_arr[0].nbytes)

        drv.memcpy_dtoh(max_end, max_end_gpu)
        
        dt =  np.float64(cfl*dx/max_end[0])  # Paso temporal inicial
        dt_dx = np.float64(dt/dx )
        T_actual = T_actual + dt
        t = np.int32(t+1)

    # Copiar los datos de vuelta al host y verificar
    data = np.empty_like(data)
    drv.memcpy_dtoh(data, data_gpu)
finally:
    # Liberar la memoria
    data_gpu.free()
    max_arr_gpu.free()
    max_end_gpu.free()
    print("Memoria liberada")

# Plot Tiempo Final

In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.linspace(0,L,n),cond_ini[:n],label='Condición Inicial')
plt.plot(np.linspace(0,L,n),data[:n],label='Solución')
plt.title("Solución")
plt.xlabel("Espacio")
plt.ylabel("Altura")
plt.show()